In [26]:
import wandb
import pandas as pd
import numpy as np
import bokeh
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, Whisker
from bokeh.transform import factor_cmap, jitter
from bokeh.layouts import gridplot

from fundusClassif.data.data_factory import get_datamodule_from_config
from nntools.utils import Config

In [27]:
api = wandb.Api()
runs = api.runs("hmr24/Grading-DiabeticRetinopathy-Comparisons-finalVersion")

config = Config('../configs/config.yaml')
datamodule = get_datamodule_from_config(config['datasets'], config['data'])
test_dataloader = datamodule.test_dataloader()
test_datasets_ids = [d.dataset.id for i, d in enumerate(test_dataloader)]
dataloader0_weight = 0
dataloader1_weight = 0
dataloader2_weight = 0
lenght_total_test = 0
for i, d in enumerate(test_dataloader):
    lenght_total_test += len(d.dataset)
#print(lenght_total_test)

for i, d in enumerate(test_dataloader):
    if d.dataset.id == test_datasets_ids[0]:
        dataloader0_weight = len(d.dataset)/lenght_total_test
        #print(dataloader0_weight)
        #print(d.dataset.id)
    elif d.dataset.id == test_datasets_ids[1]:
        dataloader1_weight += len(d.dataset)/lenght_total_test
        #print(dataloader1_weight)
        #print(d.dataset.id)
    elif d.dataset.id == test_datasets_ids[2]:
        dataloader2_weight += len(d.dataset)/lenght_total_test
        #print(dataloader2_weight)
        #print(d.dataset.id)


important_keys = ['lr','preprocessing', 'data_augmentation_type', "architecture"]
list_configs = []
for i, r in enumerate(runs):
        kappa = r.summary_metrics.get("Validation Quadratic Kappa", None)
        kappa_ddr = r.summary_metrics.get("Quadratic Kappa_DDR_test", None)
        kappa_eyepacs = r.summary_metrics.get("Quadratic Kappa_EYEPACS_test", None)
        kappa_idrid = r.summary_metrics.get("Quadratic Kappa_IDRID_test", None)
        preprocessing = r.config.get("data_preprocessing/name", None)
        architecture = r.config.get("model/architecture",None)

        if kappa is None:
            continue
        config = {k: v for k, v in r.config.items() if k in important_keys}
        config['name'] = r.name
        config['kappa'] = kappa
        config['kappa_ddr'] = kappa_ddr
        config['kappa_eyepacs'] = kappa_eyepacs
        config['kappa_idrid'] = kappa_idrid
        config['preprocessing'] = preprocessing
        config['architecture'] = architecture

        
    
        list_configs.append(config)

df = pd.DataFrame(list_configs)

df['kappa_weighted'] = 0 
for i in range(len(df)):
    eyepacs = df['kappa_eyepacs'].iloc[i]*dataloader0_weight
    idrid = df['kappa_idrid'].iloc[i]*dataloader1_weight
    ddr = df['kappa_ddr'].iloc[i]*dataloader2_weight
    df['kappa_weighted'].iloc[i] =  eyepacs + idrid + ddr
df

/home/florian/miniconda3/envs/DL/lib/python3.12/site-packages/nntools/dataset/classif_dataset.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values(self.file_column, inplace=True)
/tmp/ipykernel_23437/3775036213.py:63: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform 

,name,kappa,kappa_ddr,kappa_eyepacs,kappa_idrid,preprocessing,architecture,kappa_weighted
0,fearless-surf-1,0.961661,0.784107,0.715827,0.731017,sarki,efficientnet_b0.ra_in1k,0.720323
1,wobbly-dawn-2,0.936828,0.702965,0.672358,0.769294,seoud,efficientnet_b0.ra_in1k,0.674535
2,easy-surf-4,0.953620,0.740053,0.718841,0.733503,autobalance,efficientnet_b0.ra_in1k,0.720256
3,fresh-blaze-5,0.956895,0.737856,0.713113,0.785852,clahe_max_green_gsc,efficientnet_b0.ra_in1k,0.714863
4,devoted-sound-6,0.961434,0.814204,0.746933,0.755625,clahe_rgb,efficientnet_b0.ra_in1k,0.751351
5,hopeful-field-7,0.951706,0.737382,0.715340,0.679211,clahe_lab,efficientnet_b0.ra_in1k,0.716718
6,smooth-plant-8,0.954006,0.759616,0.718597,0.748497,absent,efficientnet_b0.ra_in1k,0.721335
7,eager-shape-9,0.946486,0.771366,0.721562,0.762782,sarki,convnextv2_nano.fcmae_ft_in22k_in1k_384,0.724896
8,crisp-wave-10,0.945966,NaN,NaN,NaN,seoud,convnextv2_nano.fcmae_ft_in22k_in1k_384,NaN
